# Lambda phage

In [ ]:
from output_helper import *
%matplotlib inline

In [ ]:
with nc.Dataset("output/lp/output_t1000.nc", "r") as ds:
    grid = grid_info(ds)
    slice_vec = np.array([0, 85, 0, 5, 10])
    P_marginal = marginalDistributions(ds, grid)
    P_sliced = slicedDistributions(ds, grid, slice_vec)
    P_marginal2D = marginalDistribution2D(ds, grid)

In [ ]:
with open("scripts/reference_solutions/lp_ssa_ref.npy", "rb") as f:
    P_marginal_ssa = np.load(f, allow_pickle=True)
    P_sliced_ssa = np.load(f, allow_pickle=True)
    P_marginal2D_ssa = np.load(f, allow_pickle=True)
    nssa = np.load(f)
    nssa_min = np.load(f)
    nssa_max = np.load(f)


In [ ]:
t = np.round(grid.t)
time_step = 10

## Figure 1
One-dimensional marginal distributions for $t = 10$

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(8, 6))
fig.delaxes(axs[-1, -1])
grid_ref = nssa
axs = plotP1Dmult(
    axs, P_marginal, P_marginal_ssa[-1], grid, grid_ref, ["DLR approximation", "SSA"])
fig.tight_layout()
fig.savefig('plots/lp_fig1.pdf')

## Figure 2
Sliced distributions for $x_0 = (0,\,0)$, $t = 500$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 6))
grid_ref = [el.shape[0] for el in P_sliced_ssa[-1]]
axs = plotP1Dmult(
    axs, P_sliced, P_sliced_ssa[-1], grid, grid_ref, ["DLR approximation", "RK45"])
fig.tight_layout()
fig.savefig('plots/ts_fig2.pdf')